In [5]:
import matplotlib.pyplot as plt
import numpy as np
import cv2

In [2]:
# 显示图像
def img_show(img):
    if len(img.shape) == 3:
        if img.shape[-1]==3:
            b,g,r = cv2.split(img)
            img = cv2.merge([r,g,b])
        elif img.shape[-1]==4:
            b,g,r,a = cv2.split(img)
            img = img
        plt.imshow(img)
    elif len(img.shape) == 2:
        plt.imshow(img,cmap="gray")

In [3]:
# 随机加椒盐函数
def salt(img,n):
    # 循环添加n个椒盐
    for k in range(n):
        # 随机添加椒盐的坐标
        i = int(np.random.random()*img.shape[0])
        j = int(np.random.random()*img.shape[1])
        # 若位灰度图
        img[i,j] = 255
        # 若为RGB图
        img[i,j,0] = 255
        img[i,j,0] = 255
        img[i,j,0] = 255
    return img

In [4]:
# 清洗锐化图片
def cleaning(img):
    bgr = cv2.split(img)
    bgrNew = []
    for c in bgr:
        c = cv2.threshold(c,127,255,cv2.THRESH_BINARY)
        bgrNew.append(c[1])
    img = cv2.merge(bgrNew)
    return img

hand = cv2.imread("/Users/tanjun/Desktop/tanjun/opencv/hand.png")
cleaning(hand)
cv2.imwrite("/Users/tanjun/Desktop/tanjun/opencv/hand.png",hand)

True

# 23.1 人脸检测

In [6]:
# 人脸检测中，主要任务是构造能够区分包含人脸实例和不包含人脸实例的分类器

### 23.1.1 基本原理

In [7]:
# 1.级联分类器
# 思路：将简单分类器按照一定顺序级联而成。（一级一级过滤分类）
# 条件1 -F-> 负类
#  ｜T
# 条件2 -F-> 负类
#  ｜T
# 条件n -F-> 负类
#  ｜T
# 正类
# 优势：开始用非常简单的判断排除明显不符合要求的实例，被排除的实例不参与下一次分类，提高分类速度

# Haar级联分类器
# opencv提供了训练好的Haar级联分类器用于人脸检测
# Haar特征：
# 垂直特征    水平特征     对角特征
#  -1 1        -1         1 -1
#               1         -1 1
# Haar特征反应的是图像的灰度变化，他将像素划分为模块后求差值。
# Haar特征用黑白两种矩形框组合成特征模版，在特征模版内，用白色矩形像素块内的像素和减去黑丝矩形像素块的像素和表示该模版的特征
# 经过上述处理，人脸特征就可以用矩形框的差值简单表示。如眼睛颜色比脸颊深，鼻梁两侧颜色比鼻梁深等
# Haar特征中的矩形框，有如下3个变量：
# 矩形位置：矩形框要逐个像素划过整个图像后去每个位置的差值
# 矩形大小：矩形大小可以任意调整
# 矩形类型：垂直，水平，对角
# 为提高效率，构造积分图，使用级联，并将Haar特征进一步划分为4类：见书P451
# 4个边特征：
# 8个线特征：
# 2个中心点特征：
# 1个对角特征：

# 除此之外，opencv还提供了Hog特征和LBP算法的级联分类器。Hog级联分类器主要用于行人检测

### 23.1.2 级联分类器的使用

In [9]:
# 训练级联分类器非常耗时，opencv提供了一些训练好的级联分类器共用户使用，这些分类器可以用来检测人脸，脸部特征，人类和其他物体
# 这些级联分类器以XML文件形式存放在opencv源文件的data目录下，加在不同级联分类器的XML文件，就可以实现不同对象的检测
# data：haarcascades,hogcascades,lbpcascades(Haarr级联分类器，HOG级联分类器，LBP级联分类器)
# Haar级联分类器多达20多种，提供了多种对象的检测功能
# XML文件名                             级联分类器类型
# haarcascade_eye.xml                  眼睛检测
# haarcascade_eye_tree_eyeglasses.xml  眼镜检测
# haarcascade_mcs_nose.xml             鼻子检测
# haarcascade_mcs_mouth.xml            嘴巴检测
# haarcascade_smile.xml                表情检测
# hogcascade_pedestrians.xml           行人检测
# lbpcasecade_frontalface.xml          正面人脸检测
# lbpcasecade_profileface.xml          人脸检测
# lbpcasecade_sliverware.xml           金属检测

# 加在级联分类器语法：
# <CascadeClassifier object> = cv2.CascadeClassifier(filename)
# faceCascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# 注意：如果通过anaconda中使用pip安装opencv，无法直接获取级联分类器xml文件，可通过两种方式获取：
# 1）安装opencv后，在安装目录下的data文件夹内查找xml文件
# 2）直接在网络上找到相应xml文件，下载并使用
# 同样，使用opencv_createsamples.exe和opencv_traincascade.exe也需要采用上述方法获取xml文件

### 23.1.3 函数介绍

In [10]:
# opencv中，人脸检测使用cv2.CascadeClassifier.detectMultiScale()函数，它可以检测出图片中所有人脸：
# object = cv2.CascadeClassifier.detectMultiScale(image[,scaleFactor[,minNeighbors[,flags[,minSize[,maxSize]]]]])
# image：待检测图像
# scaleFactor：在前后两次扫描中，搜索窗口的缩放比例
# minNeighbors：构成检测目标的相邻矩形的最小数目，默认为3，意味着3个以上的检测标记存在时才认定为人脸存在
# flags：同常被省略
# minSize：目标的最小尺寸
# maxSize：目标的最大尺寸
# objects：目标对象的矩形框向量组

### 23.1.4 案例介绍

In [18]:
wzx = cv2.imread("/Users/tanjun/Desktop/tanjun/opencv/wzx.jpg")
gray = cv2.cvtColor(wzx,cv2.COLOR_BGR2GRAY)

# 加在xml人脸检测器
faceCascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# 调用函数detectMultiScale
faces = faceCascade.detectMultiScale(gray,scaleFactor=1.15,minNeighbors=5,minSize=(5,5))

# 逐个标注人脸
for (x,y,w,h) in faces:
#     cv2.rectangle(wzx,(x,y),(x+w,y+h),(0,0,255),2)  # 矩形标注
    cv2.rectangle(wzx,(int((x+x+w)/2)),int((y+y+h)/2),int(w/2),(0,255,0),2)  # 圆形标注
    
# 显示结果
img_show(wzx)

# 23.2 LBPH人脸识别

In [19]:
# opencv提供了3种人脸识别方法：LBPH，EigenFishfaces，Fisherfaces
# LBPH（局部二值模式直方图）：基于LBP算法。LBP最早被作为纹理描述算子提出，由于在描述图像局部纹理特征效果出众得到广泛应用

### 23.2.1 基本原理

In [20]:
# LBP算法的基本原理是，将像素点A的值与邻近的8个像素点值比较：
# 如果A的值大于其邻近的像素点值，则得到0
# 如果A的值小于其邻近的像素点值，则得到1
# 最后，将像素点A与其周围的0个像素点比较得到的0，1值连起来，得到8位二进制序列，将二进制转换为十进制数作为A的LBP值

# 例如：
# 76与周围8点比较，得到二值图，再以任意一点开始，组成二值数（如从正上方开始，顺时针排列），再将二值数转十进制作为中心点的LBP值
# 128 36  251     1 0 1
# 48  76  9    >  0   0  > 01011001  > 89
# 11  213 99      0 1 1
# 对图像逐个像素用以上方式处理，得到LBP特征图想，这个特征图像的直方图称为LBPH，或称LBP直方图

# 为了得到不同尺度下的纹理结构，还可使用圆形邻域，将计算扩大到任意大小的邻域内
# 圆形邻域可用（P，R）表示，P表示圆形邻域内参与计算的像素点个数，R表示邻域的半径

# 人脸的整体灰度由于首光线影响会经常发生变化，但人脸各部分间的相对灰度基本保持一致，LBP的主要思想就是用相对关系进行处理

### 23.2.2 函数介绍

In [21]:
# opencv中使用函数cv2.face.LBPHFaceRecognizer_create()生成LBPH识别器实例模型
# 然后用cv2.face_FaceRecognizer.train()完成训练
# 最后用cv2.face_FaceRecognizer.predict()完成人脸识别

# 1.cv2.face.LBPHFaceRecognizer_create()
# retval = cv2.face.LBPHFaceRecognizer_create([,radius[,neighbors[,grid_x[,grid_y[,threshold]]]]])
# radius：半径，默认1
# neighbors：邻域点个数，默认8
# grid_x：将LBP特征图划分为一个个单元格时，每个单元格在水平方向上的像素个数，默认8，即在行方向以8个像素为单位分组
# grid_y：将LBP特征图划分为一个个单元格时，每个单元格在竖直方向上的像素个数，默认8，即在列方向以8个像素为单位分组
# threshold：在预测时使用的阈值。如果大于该阈值，就认为没有识别到任何目标对象

# 2.cv2.face_Face_Recognizer.train()
# None = cv2.face_Face_Recognizer.train(src,labels)
# src：训练图像，用于训练的人脸图像
# labels：标签，人脸图所对应的标签

# 3.cv2.face_FaceRecognizer.predict()
# label,confidence = cv2.face_FaceRecognizer.predict(src)
# src：需要识别的人脸图像
# label：返回识别结果标签
# confidence：置信度评分。衡量识别结果与原有模型间的距离，0表示完全匹配，通常50一下可以接受，大于80差别较大

### 23.2.3 案例介绍

In [22]:
# 实现一简单人脸识别

# 训练集
images = []
images.append(cv2.imread('a1.png',cv2.IMREAD_GRAYSCALE))
images.append(cv2.imread('a2.png',cv2.IMREAD_GRAYSCALE))
images.append(cv2.imread('b1.png',cv2.IMREAD_GRAYSCALE))
images.append(cv2.imread('b2.png',cv2.IMREAD_GRAYSCALE))

# 训练集标签
labels = [0,0,1,1]

# 测试图片
predict_iamge = cv2.imread('a3.png',cv2.IMREAD_GRAYSCALE)

recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.train(images,np.array(labels))
label,confidence = recognizer.predict(predict_iamge)

print(label,confidence)

# 23.3 EigenFace人脸识别

In [24]:
# EigenFace通常被称为特征脸，它使用主成分分析PCA方法将高维人脸数据处理称低维后进行数据分析，获取识别结果

### 23.3.1 基本原理

In [25]:
# 抽去主成分，如圆的半径，直径，周长，面积，都可以用半径表示，所以半径就是主成分

### 23.3.2 函数介绍

In [26]:
# opencv中使用函数cv2.face.EigenFaceRecognizer_create()生成LBPH识别器实例模型
# 然后用cv2.face_FaceRecognizer.train()完成训练
# 最后用cv2.face_FaceRecognizer.predict()完成人脸识别

# 1.cv2.face.EigenFaceRecognizer_create()
# retval = cv2.face.EigenFaceRecognizer_create([,num_components[,threshold]])
# num_components：在PCA中要保留的分量个数，一般80个分量足够
# threshold：人脸识别时采用的阈值

# 2.cv2.face_Face_Recognizer.train()
# None = cv2.face_Face_Recognizer.train(src,labels)
# src：训练图像，用于训练的人脸图像
# labels：标签，人脸图所对应的标签

# 3.cv2.face_FaceRecognizer.predict()
# label,confidence = cv2.face_FaceRecognizer.predict(src)
# src：需要识别的人脸图像
# label：返回识别结果标签
# confidence：置信度评分。衡量识别结果与原有模型间的距离，0表示完全匹配，该参数通常在0-20000之间，低于5000则认为相当可靠

### 23.3.3 案例介绍

In [27]:
# 训练集
images = []
images.append(cv2.imread('a1.png',cv2.IMREAD_GRAYSCALE))
images.append(cv2.imread('a2.png',cv2.IMREAD_GRAYSCALE))
images.append(cv2.imread('b1.png',cv2.IMREAD_GRAYSCALE))
images.append(cv2.imread('b2.png',cv2.IMREAD_GRAYSCALE))

# 训练集标签
labels = [0,0,1,1]

# 测试图片
predict_iamge = cv2.imread('a3.png',cv2.IMREAD_GRAYSCALE)

recognizer = cv2.face.EigenFaceRecognizer_create()
recognizer.train(images,np.array(labels))
label,confidence = recognizer.predict(predict_iamge)

print(label,confidence)

# 23.4 Fisherface人脸识别

In [28]:
# PCA是EigenFace方法的核心，它找到了最大化数据总方差特征的线性组合。但缺点是缺失了一部分信息
# Fisherface采用LDA（线性判断分析）实现人脸识别。

### 23.4.1 基本原理

In [29]:
# 线性判断分析在对特征降为同时考虑类别信息。
# 思路：在低维表示下，相同的类应该紧密地聚集在一起，不同的类别应该尽量分开：
# 类别间差别尽可能大
# 类别内差别尽可能小

# 做线性判别分析时，首先将训练样本集投射到一条直线A上，让投影后的点满足：
# 同类间的点尽可能地靠近
# 异类间的点尽可能地满足
# 做完投影后，经待测样本投影到直线A上，根据投影点的位置判定样本类别，就完成了识别

# 线性判别分析就是要找到一条最优的投影点

### 23.4.2 函数介绍

In [30]:
# opencv中使用函数cv2.face.FisherFaceRecognizer_create()生成LBPH识别器实例模型
# 然后用cv2.face_FaceRecognizer.train()完成训练
# 最后用cv2.face_FaceRecognizer.predict()完成人脸识别

# 1.cv2.face.FisherFaceRecognizer_create()
# retval = cv2.face.FisherFaceRecognizer_create([,num_components[,threshold]])
# num_components：保留的成分数量，可用默认值0，让函数自动设置合适的成分数量
# threshold：人脸识别时采用的阈值，如果最近的距离比预设的阈值还要大，函数返回-1

# 2.cv2.face_Face_Recognizer.train()
# None = cv2.face_Face_Recognizer.train(src,labels)
# src：训练图像，用于训练的人脸图像
# labels：标签，人脸图所对应的标签

# 3.cv2.face_FaceRecognizer.predict()
# label,confidence = cv2.face_FaceRecognizer.predict(src)
# src：需要识别的人脸图像
# label：返回识别结果标签
# confidence：置信度评分。衡量识别结果与原有模型间的距离，0表示完全匹配，该参数通常在0-20000之间，低于5000则认为相当可靠

In [31]:
# 训练集
images = []
images.append(cv2.imread('a1.png',cv2.IMREAD_GRAYSCALE))
images.append(cv2.imread('a2.png',cv2.IMREAD_GRAYSCALE))
images.append(cv2.imread('b1.png',cv2.IMREAD_GRAYSCALE))
images.append(cv2.imread('b2.png',cv2.IMREAD_GRAYSCALE))

# 训练集标签
labels = [0,0,1,1]

# 测试图片
predict_iamge = cv2.imread('a3.png',cv2.IMREAD_GRAYSCALE)

recognizer = cv2.face.FisherFaceRecognizer_create()
recognizer.train(images,np.array(labels))
label,confidence = recognizer.predict(predict_iamge)

print(label,confidence)

# 23.5 人脸数据库

In [33]:
# 1.CAS-PEAL
# 2.AT&T Facedatabase
# 3.Yale Facedatabase A
# 4.Extend .Yale Facedatabase B
# 5.color FERER database
# 6.人脸数据库整理网站：http://face-rec.org/databases/